# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# просмотр общей информации
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Имеем 12 столбцов с названиями укладывающимися в нормы. Имеются пропуски в столбцах days_employed и total_incom. Данные в виде вещественных чисел, целых чисел и строк.**

In [2]:
# просмотр первых десяти строк
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [48]:
#функция для вывода данных
def datai(dataframes):
    dataframes.columns = [x.lower().replace(' ', '_') for x in dataframes.columns.values]
    for column in dataframes.columns:
        print('====================')
        print(column)
        print(dataframes[column].value_counts())
        print(dataframes[column].unique())
    print(dataframes.info())
    display(dataframes.head())

In [49]:
datai(data)

index
0        1
19779    1
11583    1
9534     1
15677    1
        ..
19100    1
4759     1
6806     1
661      1
2047     1
Name: index, Length: 21454, dtype: int64
[    0     1     2 ... 21522 21523 21524]
children
0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64
[1 0 3 2 4 5]
days_employed
2194     2109
133        16
327        16
438        15
223        14
         ... 
5211        1
8247        1
51          1
16427       1
2049        1
Name: days_employed, Length: 7787, dtype: int64
[ 8437  4024  5623 ...   960  9929 14330]
dob_years
35    616
40    607
41    605
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    537
37    536
48    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
53    459
57    456
58    454
51    446
59    443
55    443
26    408
60    374
25    357
61    354
62    348
63    269
24    264
64  

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category,income_group,number_child
0,0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилищный кредит,Зачем им кредит?,1-2 ребенка
1,1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автокредит,ниже среднего,1-2 ребенка
2,2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилищный кредит,средний,бездетный
3,3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,кредит на образование,Зачем им кредит?,3 и более детей
4,4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,потребительский кредит,средний,бездетный


In [10]:
# заменяем XNA на F
data['gender'] = data['gender'].replace('XNA', 'F')

In [15]:
# замена значения 20 на 2 в столбце children
data['children'] = data['children'].replace(20, 2)

In [16]:
# проверка замены
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

В данных имеются пропуски (столбцы days_employed и total_income), дублирующие категории в столбце образование, ошибочные значения в столбце дети, данные в столбце по целям кредит необходимо укрупнить в категории, также произведена замена ошибки в выборе пола. С циклом не вышло. Пишет ошибку в четвертой строке.

##  Предобработка данных

### Обработка пропусков

In [18]:
# приводим значения стажа (столбец days_employed) в абсолютные значения и переводим значения выраженные в часах в дни
data['days_employed'] = data['days_employed'].apply(abs)
data.loc[data['days_employed'] >30000, 'days_employed'] = data.loc[data['days_employed'] >30000, 'days_employed'] / 24

In [19]:
# считаем количество пропусков в столбце days_employed
'Пропуски до:', data['days_employed'].isna().sum()

('Пропуски до:', 2174)

In [20]:
# заменяем пустые значения на медиану и проверяем все ли заполнено
data.loc[data['days_employed'].isna(), 'days_employed'] = data['days_employed'].median()
'Пропуски после:', data['days_employed'].isna().sum()

('Пропуски после:', 0)

In [21]:
# по аналогии делаем со столбцом total_income
'Пропуски до:', data['total_income'].isna().sum()

('Пропуски до:', 2174)

In [22]:
data.loc[data['total_income'].isna(), 'total_income'] = data['total_income'].median()
'Пропуски после:', data['total_income'].isna().sum()

('Пропуски после:', 0)

Пропуски в стаже и в заработке - это вероятно безработные люди и несовершеннолетние. Следовательно и заполнение медианой не совсем корректно... Быть может стоило заполнить нулями. Обработали пропуски заменив их на медиану столбцов total_income и days_employed

### Замена типа данных

In [23]:
# переводим значения в столбцах days_employed и total_income в целочисленный вид
data['days_employed'] = data['days_employed'].astype(int)

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [25]:
# переводим значения в столбцах days_employed и total_income в целочисленный вид
data['total_income'] = data['total_income'].astype(int)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Выбрал метод astype(), поскольку to_numeric() переводит из строковых значений в вещественные числа**

### Обработка дубликатов

In [27]:
# смотрим количество уникальных значений столбца education
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [28]:
# переводим названия категорий в строчный вид
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [29]:
# находим "полнострочные" дубликаты
data[data.duplicated(keep = False)].sort_values(by = list(data.columns))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,2194,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
19321,0,2194,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
3452,0,2194,29,высшее,0,женат / замужем,0,M,сотрудник,0,145017,покупка жилой недвижимости
18328,0,2194,29,высшее,0,женат / замужем,0,M,сотрудник,0,145017,покупка жилой недвижимости
4216,0,2194,30,среднее,1,женат / замужем,0,M,сотрудник,0,145017,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,2194,34,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для сдачи
9013,2,2194,36,высшее,0,женат / замужем,0,F,госслужащий,0,145017,получение образования
14432,2,2194,36,высшее,0,женат / замужем,0,F,госслужащий,0,145017,получение образования
11033,2,2194,39,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу


In [30]:
# удаляем дубликаты
data = data.drop_duplicates().reset_index()

In [31]:
'Дубликаты после:', data.duplicated().sum()

('Дубликаты после:', 0)

**дубликаты в образовании объединили путем перевода в строчный вид. Полные дубликаты строк удалили обычным способом drop_duplicates(). Причины появления дубликатов могут быть в объединении нескольких баз, человеческим фактором**

### Лемматизация

In [32]:
# импортируем pymystem3
from pymystem3 import Mystem
m = Mystem() 
# лемматизируем столбец цель кредита
lemmas = m.lemmatize(' '.join(data['purpose']))

In [33]:
# подсчитываем количество упоминаний слов в заявленных целях на кредит
from collections import Counter
Counter(lemmas)

Counter({'покупка': 5897,
         ' ': 55023,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

In [34]:
# объединяем похожие цели в категории
# создаем список категорий
category = ['жилищный кредит', 'автокредит', 'кредит на образование', 'потребительский кредит']
# пишем функцию которая возвращает категорию по значению purpose
def loans(categories):
    lem = m.lemmatize(categories)
    if 'жилье' in lem:
        return 'жилищный кредит'
    if 'недвижимость' in lem:
        return 'жилищный кредит'
    if 'автомобиль' in lem:
        return 'автокредит'
    if 'образование' in lem:
        return 'кредит на образование'
    return 'потребительский кредит'
# создаем новый столбец category 
data['category'] = data['purpose'].apply(loans)

In [35]:
# выводим результат
data.head(10)

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category
0,0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилищный кредит
1,1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автокредит
2,2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилищный кредит
3,3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,кредит на образование
4,4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,потребительский кредит
5,5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилищный кредит
6,6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилищный кредит
7,7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,кредит на образование
8,8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,потребительский кредит
9,9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилищный кредит


In [36]:
data['category'].count()

21454

**Данные лемматизированы, объединены в понятные группы по целям кредита. Большая часть кредитов взята на жилье и недвижимость.**

### Категоризация данных

In [37]:
# выделяем категории заемщиков по семейному статусу
data[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id')

,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


In [38]:
# выделяем категории заемщиков по образованию
data[['education_id', 'education']].drop_duplicates().set_index('education_id')

,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


In [39]:
# решили делать по медиане
# создаем функцию для разделения столбца заработка на категории
def income_group(income):
    if income <= 75000:
        return 'низкий'
    if income <= 125000:
        return 'ниже среднего'
    if income <= 175000:
        return 'средний'
    if income <= 215000:
        return 'выше среднего'
    return 'Зачем им кредит?'
data['income_group'] = data['total_income'].apply(income_group)
data.head(10)

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category,income_group
0,0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилищный кредит,Зачем им кредит?
1,1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автокредит,ниже среднего
2,2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилищный кредит,средний
3,3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,кредит на образование,Зачем им кредит?
4,4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,потребительский кредит,средний
5,5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилищный кредит,Зачем им кредит?
6,6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилищный кредит,Зачем им кредит?
7,7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,кредит на образование,средний
8,8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,потребительский кредит,ниже среднего
9,9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилищный кредит,средний


In [40]:
# смотрим сумму уникальных значений столбца. Наверное, разделение произзошло пропорционально верно, 
# ниже среднего около 6 тысяч и выше среднего около 6 тысяч.Хм... Ну это потому что по медиане делил:)
data['income_group'].value_counts()

средний             7308
ниже среднего       5504
Зачем им кредит?    4269
выше среднего       2508
низкий              1865
Name: income_group, dtype: int64

In [41]:
# выводим сумму каждой категории
data['category'].value_counts()

жилищный кредит           10811
автокредит                 4306
кредит на образование      4013
потребительский кредит     2324
Name: category, dtype: int64

In [42]:
number_child = ['бездетный', '1-2 ребенка', '3 и более детей']
# пишем функцию которая возвращает категорию по значению purpose
def child(children):
    if children == 0:
        return 'бездетный'
    if children <= 2:
        return '1-2 ребенка'
    return '3 и более детей'
# создаем новый столбец category 
data['number_child'] = data['children'].apply(child)

In [43]:
# выводим сумму каждой категории
data['number_child'].value_counts()

бездетный          14091
1-2 ребенка         6983
3 и более детей      380
Name: number_child, dtype: int64

**Для последующего анализза выделили категории по детям, образованию и семейному положению**

##  Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [44]:
# создаем табличку с расчитанной конверсией с разделением по количеству детей
table_child_debt = data.pivot_table(index = 'number_child', columns = 'debt', values = 'total_income', aggfunc = 'count')
table_child_debt.columns = ['no_debt', 'debt']
table_child_debt['%'] = (table_child_debt['debt'] / (table_child_debt['debt'] + table_child_debt['no_debt']))*100
table_child_debt.round(2)

,no_debt,debt,%
number_child,,,
1-2 ребенка,6336,647,9.27
3 и более детей,349,31,8.16
бездетный,13028,1063,7.54


**Видна прямая зависимость вероятности невозврата кредита от количества детей в семье. Хотя стоит заметить, что при трех детях почему то просрочек меньше, что вероятно объясняется небольшой выборкой**

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [45]:
# создаем табличку с расчитанной конверсией с разделением по семейному статусу
table_family_debt = data.pivot_table(index = 'family_status', columns = 'debt', values = 'total_income', aggfunc = 'count')
table_family_debt.columns = ['no_debt', 'debt']
table_family_debt['%'] = (table_family_debt['debt'] / (table_family_debt['debt'] + table_family_debt['no_debt']))*100
table_family_debt.round(2)

,no_debt,debt,%
family_status,,,
Не женат / не замужем,2536,274,9.75
в разводе,1110,85,7.11
вдовец / вдова,896,63,6.57
гражданский брак,3763,388,9.35
женат / замужем,11408,931,7.55


**Хуже всего отадют кредиты незамужние и живущие в гражданском браке, что вероятно объясняется молодостью и невысокой зарплатой, с другой стороны люди в разводе и вдовцы реже допускают просрочки (люди старше и с более высокой зарплатой).**

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [46]:
# создаем табличку с расчитанной конверсией с разделением по уровню заработка
table_income_debt = data.pivot_table(index = 'income_group', columns = 'debt', values = 'total_income', aggfunc = 'count')
table_income_debt.columns = ['no_debt', 'debt']
table_income_debt['%'] = (table_income_debt['debt'] / (table_income_debt['debt'] + table_income_debt['no_debt']))*100
table_income_debt.round(2)

,no_debt,debt,%
income_group,,,
Зачем им кредит?,3970,299,7.00
выше среднего,2299,209,8.33
ниже среднего,5032,472,8.58
низкий,1729,136,7.29
средний,6683,625,8.55


**Люди с самым высоким уровнем дохода, как и с самым низким допускают меньше просрочек, нежели прочие. Так что судить по уровню дохода о вероятности просрочки все же не стоит**

- Как разные цели кредита влияют на его возврат в срок?

In [47]:
# создаем табличку с расчитанной конверсией с разделением по цели кредита
table_income_debt = data.pivot_table(index = 'category', columns = 'debt', values = 'total_income', aggfunc = 'count')
table_income_debt.columns = ['no_debt', 'debt']
table_income_debt['%'] = (table_income_debt['debt'] / (table_income_debt['debt'] + table_income_debt['no_debt']))*100
table_income_debt.round(2)

,no_debt,debt,%
category,,,
автокредит,3903,403,9.36
жилищный кредит,10029,782,7.23
кредит на образование,3643,370,9.22
потребительский кредит,2138,186,8.00


**Жилищный кредит наименее рисковый судя по нашей конверсии**

## Общий вывод

Вероятность просрочки по кредитам увеличивается при наличии детей и молодого возраста (холостые и находящиеся в гражданском браке). Судить о вероятности просрочки исходя из уровня заработка все таки не стоит, поскольку результаты конверсии противоречивые. Воззможно данные по суммам кредитов несколько прояснили и показали зависимость заработка и просрочки.